# Week6 - Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [4]:
df.isnull().sum()

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
data = df.fillna(df.mean())

C:\Users\RESHMA CHOWDARY\AppData\Local\Temp\ipykernel_13020\3475002019.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data = df.fillna(df.mean())


In [6]:
df.dropna(subset=['Gender','Self_Employed','Dependents'], inplace=True)

In [7]:
data.isnull().sum()

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
df['Loan_Status'].value_counts()

Y    249
N     98
Name: Loan_Status, dtype: int64

In [9]:
numerical_vars = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']
categorical_vars = ['Loan_ID','Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']

In [10]:
x = df[numerical_vars + categorical_vars]
y = df['Loan_Status']

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=120)

In [12]:
categorical_vars = ['Loan_ID','Gender','Married','Dependents','Education','Self_Employed','Property_Area']

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder  


numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('standardize_num', StandardScaler())
])


In [14]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

In [15]:
numerical_data = numerical_pipeline.fit_transform(x_train[numerical_vars])
numerical_data .shape

(277, 5)

In [16]:
categorical_vars = ['Loan_ID','Gender','Married','Dependents','Education','Self_Employed','Property_Area']

In [17]:
categorical_data = categorical_pipeline.fit_transform(x_train[categorical_vars])
print('Categorical data shape:',categorical_data.shape)

Categorical data shape: (277, 285)


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_vars),
    ('cat', categorical_pipeline, categorical_vars)
])

In [30]:

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression())
])

In [31]:
lr_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize_num',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Loan_ID', 'Gender',
                                                   'Married', 'Dependents',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('logreg', LogisticRegression())])

In [32]:
train_acc = lr_pipeline.score(x_train, y_train)
test_acc = lr_pipeline.score(x_test, y_test)
print('Training accuracy:', train_acc)
print('Test accuracy:', test_acc)

Training accuracy: 0.7906137184115524
Test accuracy: 0.7142857142857143


C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [72]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__num__standardize_num__with_mean': [True, False],
    'preprocessor__num__standardize_num__with_std': [True, False],
    'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}

In [73]:
gcv_results = GridSearchCV(estimator=lr_pipeline, param_grid=param_grid, scoring='accuracy')
gcv_results = gcv_results.fit(x_train, y_train)

C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\RESHMA CHOWDARY\anaconda3\Lib\site-packages\sklearn

In [75]:
gcv_results.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_logreg__C', 'param_preprocessor__num__imputer__strategy', 'param_preprocessor__num__standardize_num__with_mean', 'param_preprocessor__num__standardize_num__with_std', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [74]:
gcv_results.best_score_

0.7038961038961039

In [76]:
gcv_results.best_params_

{'logreg__C': 1,
 'preprocessor__num__imputer__strategy': 'mean',
 'preprocessor__num__standardize_num__with_mean': True,
 'preprocessor__num__standardize_num__with_std': True}

In [77]:
gcv_results.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('standardize_num',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Loan_ID', 'Gender',
                                                   'Married', 'Dependents',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('logreg', LogisticRegression(C=1))])